# Solving the Dice Game Pig with Dynamic Programming
*This notebook is going to be updated frequently, stay tuned*.

We're going to define the game state as a Markov Process, where each game state is as follows: [player_score, opponent_score, turn_total].

In [291]:
import random

class Pig:
    def __init__(self):
        self.state = [0, 0, 0]
        self.current_player = 1
        self.player_names = ("Player 1", "Player 2")
        
    def roll(self):
        dice_value = random.randint(1, 6)
        if dice_value == 1:
            self.state[2] = 0
        else:
            self.state[2] += dice_value
        return dice_value
        
    def hold(self):
        self.state[0] += self.state[2]
        self.state[2] = 0
        return 0  # 0 indicates hold
    
    def change_player(self):
        self.state[0], self.state[1] = self.state[1], self.state[0]
        self.current_player = 3 - self.current_player
    
    def action(self, action):
        assert action in [0, 1], "Invalid move selected."  
        assert not self.is_done(), "Game is over. Actions not possible."
            
        if action == 0:
            roll_value = self.hold()
        elif action == 1:
            roll_value = self.roll()
            
        info = self.get_info(action, roll_value)
        
        if roll_value in [0, 1]:  # if hold or roll a 1
            if self.state[0] < 100:
                self.change_player()
        
        observations = self.get_observations()
        reward = self.calculate_reward()
        done = self.is_done()
        
        return observations, reward, done, info
        
    def get_observations(self):
        return self.state.copy()
    
    def get_actions(self):
        return [] if self.is_done() else [0, 1]
            
    def reset(self):
        self.state = [0, 0, 0]
        self.current_player = 1
        return self.get_observations()
    
    def calculate_reward(self):
        if self.state[0] >= 100:
            return 1
        elif self.state[1] >= 100:
            return -1
        else:
            return 0
        
    def get_info(self, action, roll_value):
        info = self.player_names[self.current_player-1]
        if action == 0:
            info += f" holds, bringing their total score to {self.state[0]}."
        elif action == 1 and roll_value != 1:
            info += f" rolls a {roll_value}, bringing their turn score to {self.state[2]} and total score to {self.state[0] + self.state[2]}."
        elif action == 1 and roll_value == 1:
            info += f" rolls a 1, losing their turn score, resetting their total score to {self.state[0] + self.state[2]}."
        return info
        
    def is_done(self):
        return self.state[0] >= 100 or self.state[1] >= 100